In [ ]:
# # Capture the installation process to avoid output clutter
# %%capture
# !pip install bertopic transformers flair gensim spacy use
# !pip install pyarrow==12.0.0 --quiet

# !pip install accelerate bitsandbytes xformers adjustText
# !pip install -U bitsandbytes

<div style='border:2px solid #2C2E4F; padding:15px'>
    NOTE: It might be best to run this on Google Colab to display the visualizations at the latter part. All insights are in the accompanying PDF file of the report.
</div>

In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic

/opt/anaconda3/envs/nlp2024/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df = pd.read_csv('/content/drive/MyDrive/AIM/NLP/df_top10_filtered-3.csv')
df = pd.read_csv('df_top10_filtered.csv')

In [3]:
# convert collected_at and metadata_event_time into datetime
df['collected_at'] = pd.to_datetime(df['collected_at'],
                                                format='%Y-%m-%d %H:%M:%S')
df['metadata_event_time'] = pd.to_datetime(df['metadata_event_time'],
                                                format='ISO8601')

<div style="background-color:#2C2E4F ; padding: 20px 0;">
    <center><h1 style="color:#E5E5E5; font-weight:bold; font-size:35px ">BEST INITIALIZATION PRACTICES</h1></center>
</div>

In [4]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") # can use SBERT
# embeddings = embedding_model.encode(df['preprocessed_text'],
#                                     show_progress_bar=True)

In [5]:
from joblib import dump, load

# Save embeddings
# dump(embeddings, 'embeddings.joblib')

In [6]:
from joblib import dump, load
loaded_embeddings = load('embeddings.joblib')
embeddings = loaded_embeddings

In [7]:
from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0,
                  metric='cosine', random_state=42)

In [8]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean',
                        cluster_selection_method='eom',
                        prediction_data=True)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english", min_df=2,
                                   ngram_range=(1, 2))

<div style="background-color:#2C2E4F ; padding: 20px 0;">
    <center><h1 style="color:#E5E5E5; font-weight:bold; font-size:35px ">LOAD LLM</h1></center>
</div>

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [11]:
from torch import cuda

model_id = 'meta-llama/Llama-2-7b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

print(device)

cpu


In [12]:
from torch import bfloat16
import transformers

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

In [ ]:
# Llama 2 Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# Llama 2 Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()

In [ ]:
# Our text generator
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

<div style="background-color:#39C5A2 ; padding: 10px 0;">
    <center><h3 style="color:#2C2E4F; font-weight:bold; font-size:25px ">Prompts for Llama 2</h3></center>
</div>

In [ ]:
# System prompt describes information given to all conversations
system_prompt = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant for labeling topics.
<</SYS>>
"""

# Example prompt demonstrating the output we are looking for
example_prompt = """
I have a topic that contains the following documents:
- Traditional diets in most cultures were primarily plant-based with a little meat on top, but with the rise of industrial style meat production and factory farming, meat has become a staple food.
- Meat, but especially beef, is the word food in terms of emissions.
- Eating meat doesn't make you a bad person, not eating meat doesn't make you a good one.

The topic is described by the following keywords: 'meat, beef, eat, eating, emissions, steak, food, health, processed, chicken'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.

[/INST] Environmental impacts of eating meat
"""

# Our main prompt with documents ([DOCUMENTS]) and keywords ([KEYWORDS]) tags
main_prompt = """
[INST]
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.
[/INST]
"""

prompt = system_prompt + example_prompt + main_prompt

In [ ]:
# System prompt describes information given to all conversations
system_prompt2 = """
<s>[INST] <<SYS>>
You are a helpful, respectful, and honest assistant for describing topics.
<</SYS>>
"""

# Example prompt demonstrating the output we are looking for
example_prompt2 = """
I have a topic that contains the following documents:
- Traditional diets in most cultures were primarily plant-based with a little meat on top, but with the rise of industrial style meat production and factory farming, meat has become a staple food.
- Meat, but especially beef, is the word food in terms of emissions.
- Eating meat doesn't make you a bad person, not eating meat doesn't make you a good one.

The topic is described by the following keywords: 'meat, beef, eat, eating, emissions, steak, food, health, processed, chicken'.

Based on the information about the topic above, please create a short description of this topic. Make sure you provide a brief explanation and nothing more.

[/INST] This topic is about the environmental impacts and health considerations related to eating meat, focusing on emissions and dietary choices.
"""

# Our main prompt with documents ([DOCUMENTS]) and keywords ([KEYWORDS]) tags
main_prompt2 = """
[INST]
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short description of this topic. Make sure you provide a brief explanation and nothing more.
[/INST]
"""

# Combine the prompts
prompt2 = system_prompt2 + example_prompt2 + main_prompt2


<div style="background-color:#2C2E4F ; padding: 20px 0;">
    <center><h1 style="color:#E5E5E5; font-weight:bold; font-size:35px ">REPRESENTATION SETUP</h1></center>
</div>

In [ ]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance,PartOfSpeech, TextGeneration

# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# Text generation with Llama 2
llama2 = TextGeneration(generator, prompt=prompt)

# Text generation with Llama 2
llama2_desc = TextGeneration(generator, prompt=prompt2)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "MMR": mmr_model,
    "Llama2": llama2,
    "Llama2_Desc": llama2_desc,
    "POS": pos_model
}

#Topic Modeling

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=15,
  verbose=True,

  # Topic Reduction
  nr_topics=10
)


In [ ]:
topics, probs = topic_model.fit_transform(df['preprocessed_text'], embeddings)

2024-08-28 18:45:05,098 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-08-28 18:45:55,756 - BERTopic - Dimensionality - Completed ✓
2024-08-28 18:45:55,759 - BERTopic - Cluster - Start clustering the reduced embeddings
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

2024-08-28 18:46:03,724 - BERTopic - Cluster - Completed ✓
2024-08-28 18:46:03,726 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 49/49 [15:20<00:00, 18.78s/it]
2024-08-28 19:08:43,406 - BERTopic - Representation - Completed ✓
2024-08-28 19:08:43,422 - BERTopic - Topic reduction - Reducing number of topics
100%|██████████| 10/10 [03:35<00:00, 21.51s/it]
2024-08-28 19:13:58,731 - BERTopic - Topic reduction - Reduced number of topics from 

In [ ]:
topic_labels = topic_model.generate_topic_labels(nr_words=15,
                                                 topic_prefix=False, separator=' - ')
topic_model.set_topic_labels(topic_labels)

In [ ]:
topic_info = topic_model.get_topic_info()

In [ ]:
topic_info

Topic  Count                                      Name  \
0     -1  15551                 -1_people_think_time_like   
1      0  10739                 0_think_people_story_like   
2      1   2985  1_country_philippine_government_filipino   
3      2   2977                2_learn_online_time_course   
4      3   1522                   3_god_sin_love_religion   
5      4    847              4_cell_animal_organism_plant   
6      5    687             5_pandemic_vaccine_people_god   
7      6    528      6_labor_obligation_employee_employer   
8      7    448            7_dance_favorite_watch_pronoun   
9      8    209         8_body_principle_workout_exercise   

                                                                                                                                                                CustomName  \
0                                                       people - think - time - like - life - way - country - work - able - know - need - thing - help - good - experience   
1                                      think - people - story - like - way - life - philosophy - art - time - experience - social - thing - different - understand - world   
2         country - philippine - government - filipino - poverty - local - people - education - english - language - national - history - development - nation - filipinos   
3                                             learn - online - time - course - class - math - mathematic - expect - work - able - like - help - thing - student - learning   
4                                         god - sin - love - religion - moral - love god - life - keenan - good - people - believe - faith - conscience - think - holiness   
5                            cell - animal - organism - plant - energy - membrane - animal cell - sep - tissue - function - specie - form - access sep - genetic - protein   
6                                    pandemic - vaccine - people - god - government - time - life - virus - vaccination - health - news - country - world - like - kingdom   
7                         labor - obligation - employee - employer - law - beer - article - worker - right - labor law - demand - code - condition - labor code - question   
8                                         dance - favorite - watch - pronoun - dancing - song - street - love - street dance - fun - music - like - netflix - class - time   
9  body - principle - workout - exercise - fitness - training - rest - stand - routine - work - principle training - recovery - individualization - principle stand - feel   

                                                                                                                                                Representation  \
0                                                       [people, think, time, like, life, way, country, work, able, know, need, thing, help, good, experience]   
1                                      [think, people, story, like, way, life, philosophy, art, time, experience, social, thing, different, understand, world]   
2         [country, philippine, government, filipino, poverty, local, people, education, english, language, national, history, development, nation, filipinos]   
3                                             [learn, online, time, course, class, math, mathematic, expect, work, able, like, help, thing, student, learning]   
4                                         [god, sin, love, religion, moral, love god, life, keenan, good, people, believe, faith, conscience, think, holiness]   
5                            [cell, animal, organism, plant, energy, membrane, animal cell, sep, tissue, function, specie, form, access sep, genetic, protein]   
6                                    [pandemic, vaccine, people, god, government, time, life, virus, vaccination, health, news, country, world, like, kingdom]   
7                         [labor, obligation, employee, employer, law, beer, article, worker, right, labor law, demand, 

<div style="background-color:#2C2E4F ; padding: 20px 0;">
    <center><h1 style="color:#E5E5E5; font-weight:bold; font-size:35px ">AUTOMATED LABELING FROM LLAMA 2</h1></center>
</div>

In [ ]:
# Initialize an empty dictionary to store the results
llama2_labels_dict = {}

# Loop over the topic indices (0 to 8 in this case)
for i in range(9):  # Assuming you have 9 topics (0 to 8)
    # Get the "Llama2" data for the current topic
    llama2_data = topic_model.get_topic(i, full=True)["Llama2"]

    # Extract the first non-empty entry (if available)
    first_label = next((label for label, count in llama2_data if label), None)

    # Add the first label to the dictionary with the topic index as the key
    llama2_labels_dict[i] = first_label

# Display the collected labels
llama2_labels_dict


{0: 'Creative Thinking and Problem Solving',
 1: 'Philippine Local Government: Balancing Autonomy and National Interest',
 2: 'Online Learning Challenges',
 3: 'God and Love',
 4: 'Cell Biology',
 5: 'Pandemic Vaccine Hesitancy',
 6: 'Labor Law and Obligations',
 7: 'Street Dance',
 8: 'Fitness Principles and Individualization'}

<div style="background-color:#2C2E4F ; padding: 20px 0;">
    <center><h1 style="color:#E5E5E5; font-weight:bold; font-size:35px ">AUTOMATED SUMMARY FROM LLAMA 2</h1></center>
</div>

In [ ]:
# Initialize an empty dictionary to store the results
llama2_labels_dict = {}

# Loop over the topic indices (0 to 8 in this case)
for i in range(9):  # Assuming you have 9 topics (0 to 8)
    # Get the "Llama2" data for the current topic
    llama2_data = topic_model.get_topic(i, full=True)["Llama2_Desc"]

    # Extract the first non-empty entry (if available)
    first_label = next((label for label, count in llama2_data if label), None)

    # Add the first label to the dictionary with the topic index as the key
    llama2_labels_dict[i] = first_label

# Display the collected labels
llama2_labels_dict


{0: 'This topic revolves around the intersection of creative thinking, science, and philosophy, exploring how these fields intersect and influence each other. The author discusses the importance of understanding the creative thinking process and how it can be applied to various disciplines, including art and science. They also delve into the concept of "the way" and how it relates to philosophy, art, and life, highlighting the significance of questioning one\'s beliefs and assumptions. Additionally, the author touches upon the role of medium in creative thinking and how it can shape our understanding of the world. Throughout the text, the author uses personal anecdotes and examples to illustrate their points, making the topic accessible and engaging for readers.',
 1: 'This topic revolves around the challenges faced by the Philippine government in addressing issues of poverty, inequality, and local governance. The discussion highlights the importance of decentralization and the role of

In [ ]:
llama2_labels = [label[0][0].split("\n")[0] for label in topic_model.get_topics(full=True)["Llama2"].values()]
topic_model.set_topic_labels(llama2_labels)

<div style="background-color:#2C2E4F ; padding: 20px 0;">
    <center><h1 style="color:#E5E5E5; font-weight:bold; font-size:35px ">VISUALIZATION</h1></center>
</div>

In [ ]:
topic_model.visualize_topics(custom_labels=True)

In [ ]:
topic_model.visualize_heatmap(custom_labels=True)

#Dynamic Topic Modeling

In [ ]:
topics_over_time = topic_model.topics_over_time(
    df['preprocessed_text'],
    df['metadata_event_time'],
    global_tuning=True,
    evolution_tuning=True,
    nr_bins=5
)

5it [00:45,  9.08s/it]


In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, custom_labels=True, top_n_topics=3)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, custom_labels=True)